In [8]:
spca_ranking = [1,0,2,5,3,6,4]
attention_ranking = [0,1,2,5,3,6,4]
corr_ranking = [1,0,2,3,5,6,4]
ig_ranking = [1,2,0,5,3,6,4]

import numpy as np
from scipy.stats import spearmanr, kendalltau

In [13]:
def rbo(a, b, p=0.9):
    # a, b are lists; p in (0,1): higher = more top-heavy persistence
    import math
    A, B = [], []
    seen_a, seen_b = set(), set()
    depth = min(len(a), len(b))
    rbo_ext = 0.0
    for d in range(1, depth+1):
        seen_a.add(a[d-1]); seen_b.add(b[d-1])
        overlap = len(seen_a & seen_b)
        rbo_ext += (overlap / d) * (p ** (d-1))
    return (1 - p) * rbo_ext
from scipy.stats import kendalltau, spearmanr

def kendall_tau_b_from_orders(a, b):
    # a, b contain same items; map to ranks then τ-b
    idx = {item:i for i,item in enumerate(a)}
    r1 = [idx[x] for x in a]
    r2 = [idx[x] for x in b]
    return kendalltau(r1, r2).correlation

def spearman_rho_from_orders(a, b):
    idx = {item:i for i,item in enumerate(a)}
    r1 = [idx[x] for x in a]; r2 = [idx[x] for x in b]
    return spearmanr(r1, r2).correlation


# print("RBO:", rbo(spca_ranking, attention_ranking))
# print("Kendall's τ-b:", kendall_tau_b_from_orders(spca_ranking, attention_ranking))
# print("Spearman's ρ:", spearman_rho_from_orders(spca_ranking, attention_ranking))
print("RBO:", rbo(spca_ranking, ig_ranking))
#print("Kendall's τ-b:", kendall_tau_b_from_orders(spca_ranking, ig_ranking))
#print("Spearman's ρ:", spearman_rho_from_orders(spca_ranking, ig_ranking))

print("RBO:", rbo(corr_ranking, ig_ranking))
print("RBO:", rbo(attention_ranking, ig_ranking))

#print("Kendall's τ-b:", kendall_tau_b_from_orders(corr_ranking, ig_ranking))
#print("Spearman's ρ:", spearman_rho_from_orders(corr_ranking, ig_ranking))

RBO: 0.47670309999999994
RBO: 0.45847809999999994
RBO: 0.37670309999999996


In [20]:
import numpy as np

def ndcg_at_4(baseline, cand, feats=7, gains="linear"):
    # baseline, cand: lists like ['f3','f1','f5','f2']
    # gains: "linear" (4,3,2,1) or "exp" (15,7,3,1)
    if gains == "linear":
        rel_map = {f: 4 - i for i, f in enumerate(baseline)}  # 4..1
    else:
        rel_map = {f: (2 ** (4 - i) - 1) for i, f in enumerate(baseline)}  # 15..1
    # candidate relevance vector in its displayed order:
    rel = np.array([rel_map.get(f, 0) for f in cand], dtype=float)
    # DCG with log2 discount
    dcg = (rel / np.log2(np.arange(2, len(cand) + 2))).sum()
    # IDCG is DCG of the baseline (its own order)
    ideal = np.array([rel_map[f] for f in baseline], dtype=float)
    idcg = (ideal / np.log2(np.arange(2, len(baseline) + 2))).sum()
    return float(dcg / idcg) if idcg > 0 else 0.0
print(ndcg_at_4(spca_ranking, ig_ranking, gains="linear"))
#print(ndcg_at_4(attention_ranking, ig_ranking, gains="linear"))
print(ndcg_at_4(corr_ranking, ig_ranking, gains="linear"))

def ndcg_at_k(baseline, cand, k=7, gains="linear"):
    """
    baseline: ideal ranking, e.g. ['f3','f1','f5','f2', ...]
    cand    : candidate ranking (same domain), e.g. ['f1','f2','f3', ...]
    k       : cutoff (NDCG@k)
    gains   : "linear" or "exp"
    """
    if k <= 0 or len(baseline) == 0:
        return 0.0

    m = min(k, len(baseline))  # how many ideal items we consider at cutoff

    # relevance for the top-m items in the baseline
    if gains == "linear":
        rel_vals = [m - i for i in range(m)]                 # m, m-1, ..., 1
    else:  # "exp"
        rel_vals = [(2 ** (m - i) - 1) for i in range(m)]    # 2^m-1, ..., 1

    rel_map = {f: r for f, r in zip(baseline[:m], rel_vals)}

    # candidate relevance vector at cutoff k
    rel = np.array([rel_map.get(f, 0) for f in cand[:k]], dtype=float)

    # DCG@k with log2 discount
    dcg = (rel / np.log2(np.arange(2, 2 + len(rel)))).sum()

    # IDCG@k: ideal order is baseline[:m]
    ideal = np.array(rel_vals, dtype=float)
    idcg = (ideal / np.log2(np.arange(2, 2 + m))).sum()

    return float(dcg / idcg) if idcg > 0 else 0.0

print(ndcg_at_k(spca_ranking, ig_ranking, gains="exp"))
#print(ndcg_at_4(attention_ranking, ig_ranking, gains="linear"))
print(ndcg_at_k(corr_ranking, ig_ranking, gains="exp"))
print(ndcg_at_k(attention_ranking, ig_ranking, gains="exp"))


0.979219452029507
0.9722639546600214
0.9782710221704757
0.9764527851303665
0.8123119698022505


In [16]:
def jaccard_at_k(a, b, k):
    A, B = set(a[:k]), set(b[:k])
    return len(A & B) / len(A | B) if A | B else 1.0

def overlap_curve_auc(a, b, k_max=None):
    n = min(len(a), len(b))
    k_max = k_max or n
    xs, ys = [], []
    for k in range(1, k_max+1):
        xs.append(k); ys.append(len(set(a[:k]) & set(b[:k])) / k)
    # trapezoidal AUC normalized to [0,1]
    import numpy as np
    auc = np.trapz(ys, xs) / k_max
    return auc, (np.array(xs), np.array(ys))
# print("Jaccard @3:", jaccard_at_k(spca_ranking, attention_ranking, 4))
# auc, (xs, ys) = overlap_curve_auc(spca_ranking, attention_ranking)
# print("Overlap Curve AUC:", auc)

print("Jaccard @4:", jaccard_at_k(spca_ranking, ig_ranking, 4))
auc, (xs, ys) = overlap_curve_auc(spca_ranking, ig_ranking)
print("Overlap Curve AUC:", auc)

print("Jaccard @4:", jaccard_at_k(corr_ranking, ig_ranking, 4))
auc, (xs, ys) = overlap_curve_auc(corr_ranking, ig_ranking)
print("Overlap Curve AUC:", auc)

print("Jaccard @4:", jaccard_at_k(attention_ranking, ig_ranking, 4))
auc, (xs, ys) = overlap_curve_auc(attention_ranking, ig_ranking)
print("Overlap Curve AUC:", auc)

Jaccard @4: 1.0
Overlap Curve AUC: 0.7857142857142857
Jaccard @4: 0.6
Overlap Curve AUC: 0.75
Jaccard @4: 1.0
Overlap Curve AUC: 0.7142857142857143


### Chosen metrics comparison

In [18]:
att = [0,1,2]
ig = [0,1,2]
spca=[0,1,2,3,5]
corr = [0,1,2,3,5]

In [19]:
def jaccard_list(a, b):
    A, B = set(a), set(b)
    if not A and not B:
        return 1.0
    return len(A & B) / len(A | B)
print(jaccard_list(att,ig))
print(jaccard_list(spca,ig))

1.0
0.6


### Overlaps of best random metrics

In [ ]:
from itertools import combinations, product

def compute_overlaps(named_sets):
    """
    named_sets: dict[str, Iterable]  e.g. {"v1": ["m1","m2"], "v2": {...}, ...}
    Returns: sizes, pairwise_intersections, all_sets_intersection, jaccard_matrix
    """
    S = {name: set(map(str, vals)) for name, vals in named_sets.items()}
    sizes = {k: len(v) for k, v in S.items()}
    all_intersection = set.intersection(*S.values()) if S else set()
    pairwise = {tuple(sorted((a, b))): (S[a] & S[b]) for a, b in combinations(S, 2)}
    jaccard = {
        a: {b: (len(S[a] & S[b]) / len(S[a] | S[b]) if (S[a] or S[b]) else 0.0) for b in S}
        for a in S
    }
    return sizes, pairwise, all_intersection, jaccard

# Example
# sets = {
#     "A": {"2","3","4","5", "6"},
#     "B": {"0","2","4", "5"},
#     "C": {"1","2","3","4"},
# }
sets = {
    "A": {"2","3","4","5", "6"},
    "B": {"0","2","4", "5"},
    
}
sizes, pairwise, all_k, jaccard = compute_overlaps(sets)

print("Sizes:", sizes)
print("All-sets intersection:", all_k)                # metrics shared by ALL sets
print("A∩B:", pairwise[("A","B")])                    # pairwise overlap
print("Jaccard(A,B):", round(jaccard["A"]["B"], 3))   # |A∩B| / |A∪B|


Sizes: {'A': 5, 'B': 4, 'C': 4}
All-sets intersection: {'2', '4'}
A∩B: {'5', '2', '4'}
Jaccard(A,B): 0.5
